# Redefining Geographies for Custom Geographies in Replcia

**What I have:** already defined searate census tracts in replica's saved selections, one for each station. 
</br>
**What I need:** combined geographies for all tracts from the already defined stations except for the station listed as origin

In [1]:
import geopandas as gpd
import pandas as pd
from siuba import *
import numpy as np

from calitp_data_analysis.sql import to_snakecase
from calitp_data_analysis import utils
from calitp_data_analysis import get_fs
import gcsfs as fs
fs = get_fs()

import os
import shutil

In [2]:
gcs_path = "gs://calitp-analytics-data/data-analyses/big_data/lossan_stations/"

In [3]:
station_list_lossan = ["lossan-simi-valley", "lossan-burbank-airport", "lossan-oxnard",  "lossan-camarillo", "lossan-moorpark",  "lossan-chatsworth",
                "lossan-northridge", "lossan-van-nuys", "lossan-burbank", "lossan-glendale", "lossan-union-station",
                "lossan-commerce", "lossan-norwalk", "lossan-buena-park",
                "lossan-fullerton", "lossan-anaheim",  "lossan-orange", "lossan-santa-ana", "lossan-tustin",
                "lossan-irvine", "lossan-laguna-niguel", "lossan-san-juan-capistrano", "lossan-san-clemente", "lossan-san-clemente-pier",
                "lossan-oceanside", "lossan-solana-beach", "lossan-san-diego", "lossan-ot-san-diego"]

In [4]:
station_list_riverside = ["riverside-line-montebello-commerce", "riverside-line-industry", "riverside-line-pomona",  "riverside-line-ontario",
                          "riverside-line-pedley",  "riverside-line-riverside", "lossan-union-station"]

In [5]:
station_list_sb = ["lossan-union-station", "sb-line-cal-state-la", "sb-line-el-monte",  "sb-line-baldwin-park",
                   "sb-line-covina",  "sb-line-pomona-north",
                         "sb-line-claremont", "sb-line-montclair", "sb-line-upland", "sb-line-rancho-cucamonga", "sb-line-fontana", 
                  "sb-line-rialto", "sb-line-san-bernardino-depot", "sb-line-san-bernardino-dt"]


In [6]:
# with get_fs().open(f"{gcs_path}saved-selection-geo-lossan-anaheim_us-tract-2020.zip") as f:
#         test = to_snakecase(gpd.read_file(f))

In [7]:
# test

In [8]:
# test.explore()

In [9]:
all_df = []

In [10]:
for station in station_list_riverside:
    with get_fs().open(f"{gcs_path}saved-selection-geo-{station}_us-tract-2020.zip") as f:
        df = to_snakecase(gpd.read_file(f))
        df['station_name'] = station
        
        all_df.append(df)

In [11]:
all_stations_riverside = pd.concat(all_df, ignore_index=True)

In [12]:
# all_stations_riverside

In [13]:
for station in station_list_sb:
    with get_fs().open(f"{gcs_path}saved-selection-geo-{station}_us-tract-2020.zip") as f:
        df = to_snakecase(gpd.read_file(f))
        df['station_name'] = station
        
        all_df.append(df)
        
all_stations_sb = pd.concat(all_df, ignore_index=True)

In [14]:
# all_stations

In [15]:
# (all_stations>>filter(_.geoname=="421.08 (Orange, CA)")).explore()

In [16]:
# (all_stations>>filter(_.geoname=="761.05 (Orange, CA)")).explore()

In [17]:
all_stations_riverside.explore()

In [28]:
# all_stations_riverside.sample()

In [18]:
### combine destination locations, aka all stations but the one at the origin 

In [18]:
print(station_list_riverside)

['riverside-line-montebello-commerce', 'riverside-line-industry', 'riverside-line-pomona', 'riverside-line-ontario', 'riverside-line-pedley', 'riverside-line-riverside', 'lossan-union-station']


In [19]:
# gcs_export = "gs://calitp-analytics-data/data-analyses/big_data/lossan_stations/station_destinations_combined/"

In [30]:
for station in station_list_riverside:
    station_destinations = all_stations_riverside>>filter(_.station_name!=station)
    
    utils.make_zipped_shapefile(station_destinations, f"station_destinations/{station}_destinations_riverside.zip")
    # station_destinations.to_file(f"{gcs_path}station_destinations_combined/{station}_destinations.shp", driver="ESRI Shapefile")
    
    print(f"Sucessfully exported {station} destinations to gcs")
    
    
### remove locations
folder_to_delete = "./station_destinations"  # Replace with the actual path to your folder

if os.path.isdir(folder_to_delete):
    try:
        shutil.rmtree(folder_to_delete)
        print(f"The folder '{folder_to_delete}' and its contents have been successfully removed.")
    except OSError as e:
        print(f"Error: {folder_to_delete} : {e.strerror}")
else:
    print(f"The folder '{folder_to_delete}' does not exist.")


Path name: station_destinations/riverside-line-montebello-commerce_destinations_riverside.zip
Dirname (1st element of path): station_destinations/riverside-line-montebello-commerce_destinations_riverside
Shapefile name: riverside-line-montebello-commerce_destinations_riverside.shp
Shapefile component parts folder: station_destinations/riverside-line-montebello-commerce_destinations_riverside/riverside-line-montebello-commerce_destinations_riverside.shp
Sucessfully exported riverside-line-montebello-commerce destinations to gcs
Path name: station_destinations/riverside-line-industry_destinations_riverside.zip
Dirname (1st element of path): station_destinations/riverside-line-industry_destinations_riverside
Shapefile name: riverside-line-industry_destinations_riverside.shp
Shapefile component parts folder: station_destinations/riverside-line-industry_destinations_riverside/riverside-line-industry_destinations_riverside.shp
Sucessfully exported riverside-line-industry destinations to gcs


/opt/conda/lib/python3.11/site-packages/calitp_data_analysis/utils.py:179: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(driver="ESRI Shapefile", filename=dirname.joinpath(shapefile_name))
/opt/conda/lib/python3.11/site-packages/calitp_data_analysis/utils.py:179: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(driver="ESRI Shapefile", filename=dirname.joinpath(shapefile_name))
/opt/conda/lib/python3.11/site-packages/calitp_data_analysis/utils.py:179: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(driver="ESRI Shapefile", filename=dirname.joinpath(shapefile_name))
/opt/conda/lib/python3.11/site-packages/calitp_data_analysis/utils.py:179: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(driver="ESRI Shapefile", filename=dirnam

Shapefile component parts folder: station_destinations/riverside-line-riverside_destinations_riverside/riverside-line-riverside_destinations_riverside.shp
Sucessfully exported riverside-line-riverside destinations to gcs
Path name: station_destinations/lossan-union-station_destinations_riverside.zip
Dirname (1st element of path): station_destinations/lossan-union-station_destinations_riverside
Shapefile name: lossan-union-station_destinations_riverside.shp
Shapefile component parts folder: station_destinations/lossan-union-station_destinations_riverside/lossan-union-station_destinations_riverside.shp
Sucessfully exported lossan-union-station destinations to gcs
The folder './station_destinations' and its contents have been successfully removed.


/opt/conda/lib/python3.11/site-packages/calitp_data_analysis/utils.py:179: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(driver="ESRI Shapefile", filename=dirname.joinpath(shapefile_name))


In [29]:
for station in station_list_sb:
    station_destinations = all_stations_sb>>filter(_.station_name!=station)
    
    station_destinations.drop_duplicates(subset=['geometry'], keep='first')
    
    utils.make_zipped_shapefile(station_destinations, f"station_destinations/{station}_destinations_sb.zip")
    # station_destinations.to_file(f"{gcs_path}station_destinations_combined/{station}_destinations.shp", driver="ESRI Shapefile")
    
    print(f"Sucessfully exported {station} destinations to gcs")
    
    
### remove locations
folder_to_delete = "./station_destinations"  # Replace with the actual path to your folder

if os.path.isdir(folder_to_delete):
    try:
        shutil.rmtree(folder_to_delete)
        print(f"The folder '{folder_to_delete}' and its contents have been successfully removed.")
    except OSError as e:
        print(f"Error: {folder_to_delete} : {e.strerror}")
else:
    print(f"The folder '{folder_to_delete}' does not exist.")





Path name: station_destinations/lossan-union-station_destinations_sb.zip
Dirname (1st element of path): station_destinations/lossan-union-station_destinations_sb
Shapefile name: lossan-union-station_destinations_sb.shp
Shapefile component parts folder: station_destinations/lossan-union-station_destinations_sb/lossan-union-station_destinations_sb.shp


/opt/conda/lib/python3.11/site-packages/calitp_data_analysis/utils.py:179: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(driver="ESRI Shapefile", filename=dirname.joinpath(shapefile_name))


Sucessfully exported lossan-union-station destinations to gcs
Path name: station_destinations/sb-line-cal-state-la_destinations_sb.zip
Dirname (1st element of path): station_destinations/sb-line-cal-state-la_destinations_sb
Shapefile name: sb-line-cal-state-la_destinations_sb.shp
Shapefile component parts folder: station_destinations/sb-line-cal-state-la_destinations_sb/sb-line-cal-state-la_destinations_sb.shp
Sucessfully exported sb-line-cal-state-la destinations to gcs
Path name: station_destinations/sb-line-el-monte_destinations_sb.zip
Dirname (1st element of path): station_destinations/sb-line-el-monte_destinations_sb
Shapefile name: sb-line-el-monte_destinations_sb.shp
Shapefile component parts folder: station_destinations/sb-line-el-monte_destinations_sb/sb-line-el-monte_destinations_sb.shp
Sucessfully exported sb-line-el-monte destinations to gcs


/opt/conda/lib/python3.11/site-packages/calitp_data_analysis/utils.py:179: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(driver="ESRI Shapefile", filename=dirname.joinpath(shapefile_name))
/opt/conda/lib/python3.11/site-packages/calitp_data_analysis/utils.py:179: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(driver="ESRI Shapefile", filename=dirname.joinpath(shapefile_name))
/opt/conda/lib/python3.11/site-packages/calitp_data_analysis/utils.py:179: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(driver="ESRI Shapefile", filename=dirname.joinpath(shapefile_name))


Path name: station_destinations/sb-line-baldwin-park_destinations_sb.zip
Dirname (1st element of path): station_destinations/sb-line-baldwin-park_destinations_sb
Shapefile name: sb-line-baldwin-park_destinations_sb.shp
Shapefile component parts folder: station_destinations/sb-line-baldwin-park_destinations_sb/sb-line-baldwin-park_destinations_sb.shp
Sucessfully exported sb-line-baldwin-park destinations to gcs
Path name: station_destinations/sb-line-covina_destinations_sb.zip
Dirname (1st element of path): station_destinations/sb-line-covina_destinations_sb
Shapefile name: sb-line-covina_destinations_sb.shp
Shapefile component parts folder: station_destinations/sb-line-covina_destinations_sb/sb-line-covina_destinations_sb.shp
Sucessfully exported sb-line-covina destinations to gcs
Path name: station_destinations/sb-line-pomona-north_destinations_sb.zip
Dirname (1st element of path): station_destinations/sb-line-pomona-north_destinations_sb
Shapefile name: sb-line-pomona-north_destinati

...

Shapefile component parts folder: station_destinations/sb-line-montclair_destinations_sb/sb-line-montclair_destinations_sb.shp
Sucessfully exported sb-line-montclair destinations to gcs
Path name: station_destinations/sb-line-upland_destinations_sb.zip
Dirname (1st element of path): station_destinations/sb-line-upland_destinations_sb
Shapefile name: sb-line-upland_destinations_sb.shp
Shapefile component parts folder: station_destinations/sb-line-upland_destinations_sb/sb-line-upland_destinations_sb.shp
Sucessfully exported sb-line-upland destinations to gcs
Path name: station_destinations/sb-line-rancho-cucamonga_destinations_sb.zip
Dirname (1st element of path): station_destinations/sb-line-rancho-cucamonga_destinations_sb
Shapefile name: sb-line-rancho-cucamonga_destinations_sb.shp


/opt/conda/lib/python3.11/site-packages/calitp_data_analysis/utils.py:179: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(driver="ESRI Shapefile", filename=dirname.joinpath(shapefile_name))
/opt/conda/lib/python3.11/site-packages/calitp_data_analysis/utils.py:179: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(driver="ESRI Shapefile", filename=dirname.joinpath(shapefile_name))
/opt/conda/lib/python3.11/site-packages/calitp_data_analysis/utils.py:179: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(driver="ESRI Shapefile", filename=dirname.joinpath(shapefile_name))


Shapefile component parts folder: station_destinations/sb-line-rancho-cucamonga_destinations_sb/sb-line-rancho-cucamonga_destinations_sb.shp
Sucessfully exported sb-line-rancho-cucamonga destinations to gcs
Path name: station_destinations/sb-line-fontana_destinations_sb.zip
Dirname (1st element of path): station_destinations/sb-line-fontana_destinations_sb
Shapefile name: sb-line-fontana_destinations_sb.shp
Shapefile component parts folder: station_destinations/sb-line-fontana_destinations_sb/sb-line-fontana_destinations_sb.shp
Sucessfully exported sb-line-fontana destinations to gcs
Path name: station_destinations/sb-line-rialto_destinations_sb.zip
Dirname (1st element of path): station_destinations/sb-line-rialto_destinations_sb
Shapefile name: sb-line-rialto_destinations_sb.shp
Shapefile component parts folder: station_destinations/sb-line-rialto_destinations_sb/sb-line-rialto_destinations_sb.shp


/opt/conda/lib/python3.11/site-packages/calitp_data_analysis/utils.py:179: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(driver="ESRI Shapefile", filename=dirname.joinpath(shapefile_name))
/opt/conda/lib/python3.11/site-packages/calitp_data_analysis/utils.py:179: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(driver="ESRI Shapefile", filename=dirname.joinpath(shapefile_name))
/opt/conda/lib/python3.11/site-packages/calitp_data_analysis/utils.py:179: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(driver="ESRI Shapefile", filename=dirname.joinpath(shapefile_name))


Sucessfully exported sb-line-rialto destinations to gcs
Path name: station_destinations/sb-line-san-bernardino-depot_destinations_sb.zip
Dirname (1st element of path): station_destinations/sb-line-san-bernardino-depot_destinations_sb
Shapefile name: sb-line-san-bernardino-depot_destinations_sb.shp
Shapefile component parts folder: station_destinations/sb-line-san-bernardino-depot_destinations_sb/sb-line-san-bernardino-depot_destinations_sb.shp
Sucessfully exported sb-line-san-bernardino-depot destinations to gcs
Path name: station_destinations/sb-line-san-bernardino-dt_destinations_sb.zip
Dirname (1st element of path): station_destinations/sb-line-san-bernardino-dt_destinations_sb
Shapefile name: sb-line-san-bernardino-dt_destinations_sb.shp
Shapefile component parts folder: station_destinations/sb-line-san-bernardino-dt_destinations_sb/sb-line-san-bernardino-dt_destinations_sb.shp
Sucessfully exported sb-line-san-bernardino-dt destinations to gcs
The folder './station_destinations' an

In [30]:
### test worked!
# shp_test = gpd.read_file("riverside-line-ontario_destinations_riverside.zip")
  